### Задание:

Вы работаете аналитиком в приложении для онлайн-знакомств. Механика приложения следующая: пользователи видят в приложении анкеты друг друга и ставят лайки или дизлайки. Если пользователи поставили друг другу лайк – это мэтч. У пользователей появляется возможность познакомиться.

Команда приложения разработала новый алгоритм для поиска наиболее подходящих анкет. Для проверки работы алгоритма провели A/B-тест. Все пользователи были разделены на две группы. Пользователи в группе с номером 0 пользовались приложением со старым алгоритмом. Пользователи в группе №1 пользовались приложением с новым алгоритмом для поиска анкет.

Ваша задача: оценить, действительно ли новый алгоритм улучшил качество сервиса. Для этого нужно выбрать метрики, которые отвечают за качество сервиса, и статистически сравнить эти метрики в двух группах.

Результат вашей работы – аналитическое заключение с ответом на вопрос, стоит ли включать новую систему поиска анкет на всех пользователей.

In [69]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [70]:
data = pd.read_csv('dating_data.csv')
data.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14514 entries, 0 to 14513
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id_1  14514 non-null  int64
 1   user_id_2  14514 non-null  int64
 2   group      14514 non-null  int64
 3   is_match   14514 non-null  int64
dtypes: int64(4)
memory usage: 453.7 KB


In [72]:
# Метрика, которую я беру за основную – конверсия в match

In [73]:
df = data.groupby(['user_id_1', 'group']).agg(count_actions=('user_id_1', 'count'), count_match=('is_match', 'sum'))
df['conversion'] = round(df['count_match']/df['count_actions'], 2)
df.head()

,,count_actions,count_match,conversion
user_id_1,group,,,
1,1,24,11,0.46
2,1,16,7,0.44
3,1,16,5,0.31
4,0,9,2,0.22
5,1,22,13,0.59


In [74]:
data_ab_test = pd.DataFrame({'group': ['A', 'B'],
                             'actions': [df.query('group == 0').count_actions.mean(), df.query('group == 1').count_actions.mean()],
                             'matches': [df.query('group == 0').count_match.mean(), df.query('group == 1').count_match.mean()],
                             'conversion': [df.query('group == 0').conversion.mean(), df.query('group == 1').conversion.mean()]
                             })

In [75]:
data_ab_test

,group,actions,matches,conversion
0,A,9.564870,1.864271,0.193593
1,B,19.482966,7.833667,0.402405


In [76]:
# Видно, что в тестовой группе конверсия увеличилась
# Также такие пользователи совершают больше действий в приложении (ставят лайки и дизлайки)
# С помощью t-теста проверим яыляется ли эти различия статистически значимыми

In [77]:
t_stat, p_value = ttest_ind(df.query('group == 0').count_actions, df.query('group == 1').count_actions, equal_var=True)
print(t_stat, p_value)

-51.85383774946492 1.8942877064043142e-285


In [78]:
# p_value < 0.05, значит кол-во действий действительно отличается

In [79]:
t_stat, p_value = ttest_ind(df.query('group == 0').conversion, df.query('group == 1').conversion, equal_var=True)
print(t_stat, p_value)

-26.472890366436975 2.0763528696725214e-117


In [80]:
# p_value < 0.05, значит конверсия действительно увеличилась

In [81]:
# Так как A/B тест показал статистически значимое увеличение конверсии в match,
# то принимается решение выкатить новое обновление для всех ползователей